PART B : Stoichometry and reaction balancing
1: Reaction Balance Function
    In this part, we will create a function that can balance a reaction equation and give the stoichometric coefficient for each molecule that balance the reaction.

In [ ]:
import re
import numpy as np
from scipy.linalg import null_space

def parse_formula(formula):
    # The objective of this function is to take a chemical formula
    # and to transform it into a dictionary of elements and their 
    # atom counts
    pattern = r'([A-Z][a-z]?)(\d*)'
    # pattern explanation: [A-Z] matches an uppercase letter and
    # [a-z]? matches an optional lowercase letter, ex: "H" or "He"
    # together they match an element symbol
    # (\d*) matches the number that follows the element 
    elements = re.findall(pattern, formula)
    # elements will be a list of tuples (element, count) throught the
    # findall function
    counts = {}
    for (element, count) in elements:
        counts[element] = counts.get(element, 0) + int(count or 1)
    #This part will count the number of atoms for each element, for 
    # each (element, count), count.get(element, 0) will return the stocked 
    # value, otherwise it is 0, int(count or 1) will convert count to integer,
    # if count is empty, it will return 1
    return counts

def Balance_reaction(reactants, products):
    # This function will balance a chemical reaction given the
    # list of reactants and products, and return the stoichiometric
    # coefficients for each molecule in the reaction
    all_elements = set()
    for formula in reactants + products:
        all_elements.update(parse_formula(formula).keys())
    all_elements = sorted(all_elements)
    # This part will put all the elements of the reactants and products
    # into a set to avoid duplicates and then we sort them
    # Example: for H2 + O2 -> H2O, all_elements will be ['H', 'O']


    n = len(all_elements)
    m = len(reactants) + len(products)
    matrix = np.zeros((n, m))
    # This part build the matrix for the system of equations where:
    # n is the number of unique elements
    # m is the number of molecules (reactants + products)
    # we initialize a nxm matrix of zeros

    for i, element in enumerate(all_elements):
        for j, formula in enumerate(reactants):
            matrix[i, j] = parse_formula(formula).get(element, 0)
        for j, formula in enumerate(products):
            matrix[i, j + len(reactants)] = -parse_formula(formula).get(element, 0)
    # This part will fill the matrix with the atom counts, 
    # for reactants we put positive counts and for products negative counts
    # The final equation system will be Ax = 0, where A is the matrix and x is the
    # stoichometric coefficients we want to find

    # Use scipy's null_space to get a solution vector
    ns = null_space(matrix)
    if ns.size == 0:
        raise ValueError("No solution found for balancing the reaction.")
    coeffs = ns[:, 0]
    coeffs = coeffs / np.min(np.abs(coeffs[np.nonzero(coeffs)]))
    coeffs = np.round(coeffs).astype(int)
    return coeffs.tolist()
    #EXPLANATION :

# Example usage:
reactants = ["H2", "O2"]
products = ["H2O"]
coefficients = Balance_reaction(reactants, products)
print("Coefficients:", coefficients)  
# Output should be [2, 1, 2] for 2H2 + 1O2 -> 2H2O



Coefficients: [2, 1, 2]
